In [1]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# initialize the initial learning rate
# INIT_LR = 1e-4
INIT_LR = 0.0001
BS = 32

DIRECTORY = r"dataset"
CATEGORIES = ["with_mask", "without_mask"]


In [3]:
data = []
labels = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)

        data.append(image)
        labels.append(category)


C:\Users\User\anaconda3\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [4]:
# convert text to Binary
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)
print(data,labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.20, stratify=labels, random_state=42)

[[[[-0.46666664 -0.49019605 -0.4352941 ]
   [-0.46666664 -0.49019605 -0.4352941 ]
   [-0.46666664 -0.49019605 -0.4352941 ]
   ...
   [-0.38823527 -0.6392157  -0.7411765 ]
   [-0.38823527 -0.6392157  -0.7411765 ]
   [-0.38823527 -0.6392157  -0.7411765 ]]

  [[-0.46666664 -0.49019605 -0.4352941 ]
   [-0.46666664 -0.49019605 -0.4352941 ]
   [-0.46666664 -0.49019605 -0.4352941 ]
   ...
   [-0.38823527 -0.6392157  -0.7411765 ]
   [-0.38823527 -0.6392157  -0.7411765 ]
   [-0.38823527 -0.6392157  -0.7411765 ]]

  [[-0.3960784  -0.4588235  -0.372549  ]
   [-0.3960784  -0.4588235  -0.372549  ]
   [-0.3960784  -0.4588235  -0.372549  ]
   ...
   [-0.45098037 -0.62352943 -0.8039216 ]
   [-0.45098037 -0.62352943 -0.8039216 ]
   [-0.45098037 -0.62352943 -0.8039216 ]]

  ...

  [[-0.4352941  -0.6862745  -0.8039216 ]
   [-0.4352941  -0.6862745  -0.8039216 ]
   [-0.4352941  -0.6862745  -0.8039216 ]
   ...
   [-0.34117645 -0.52156866 -0.6313726 ]
   [-0.34117645 -0.52156866 -0.6313726 ]
   [-0.34117645 

In [5]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [6]:
# load the MobileNetV2 network
baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

In [7]:
# Create head and the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [8]:
# Call head and the base model
model = Model(inputs=baseModel.input, outputs=headModel)

In [9]:
# loop over all layers in the base model and freeze them
for layer in baseModel.layers:
    layer.trainable = False

In [10]:
# compile our model 17
model.compile(loss="binary_crossentropy", optimizer='adam',
    metrics=["accuracy"])

In [11]:
# train the head of the network
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=10)

Epoch 1/10
95/95 [==============================] - 108s 1s/step - loss: 0.1519 - accuracy: 0.9387 - val_loss: 0.0492 - val_accuracy: 0.9870
Epoch 2/10
95/95 [==============================] - 92s 965ms/step - loss: 0.0484 - accuracy: 0.9835 - val_loss: 0.0336 - val_accuracy: 0.9909
Epoch 3/10
95/95 [==============================] - 91s 957ms/step - loss: 0.0403 - accuracy: 0.9895 - val_loss: 0.0265 - val_accuracy: 0.9922
Epoch 4/10
95/95 [==============================] - 95s 996ms/step - loss: 0.0364 - accuracy: 0.9871 - val_loss: 0.0268 - val_accuracy: 0.9909
Epoch 5/10
95/95 [==============================] - 100s 1s/step - loss: 0.0363 - accuracy: 0.9898 - val_loss: 0.0313 - val_accuracy: 0.9922
Epoch 6/10
95/95 [==============================] - 94s 992ms/step - loss: 0.0325 - accuracy: 0.9895 - val_loss: 0.0251 - val_accuracy: 0.9922
Epoch 7/10
95/95 [==============================] - 97s 1s/step - loss: 0.0322 - accuracy: 0.9888 - val_loss: 0.0225 - val_accuracy: 0.9935
Epoch 

In [12]:
# make predictions on the testing set
predIdxs = model.predict(testX, batch_size=BS)
predIdxs = np.argmax(predIdxs, axis=1)

Network evaluation...
24/24 [==============================] - 19s 727ms/step


In [13]:
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
    target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.98      1.00      0.99       383
without_mask       1.00      0.98      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767



In [14]:
# plot the training loss and accuracy
# N = EPOCHS
# plt.style.use("ggplot")
# plt.figure()
# plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
# plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
# plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
# plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
# plt.title("Training Loss and Accuracy")
# plt.xlabel("Epoch #")
# plt.ylabel("Loss/Accuracy")
# plt.legend(loc="lower left")
# plt.savefig("plot.png")

In [15]:
# serialize the model to disk
model.save("mask_detector2.model", save_format="h5")